### 다중언어 개체명인식

- 제로샷 교차 언어 전이 (zero-shot cross lingual switching) 가능
  - 한 언어에서 파인 튜닝된 모델이 훈련없이 다른 모델에 적용 가능하다.
- 코드 스위칭 에 적합
  - 하나의 대화에서 둘이상의 언어나, 사투리등을 바꿈

- XTREME 데이터 셋 이용
    - PAN-X, wikiANN : 교차 언어 데이터 셋
    - LOC, PER, ORG
    - IOB2 포맷
        - i- 동일 개체명의 연속
        - o- 어떤 개체에도 속하지 않음
        - B- 개체명의 시작

In [ ]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
"서브셋 갯수",len(xtreme_subsets)

('서브셋 갯수', 183)

In [85]:
for i,l in enumerate(xtreme_subsets):
    if "PAN" in l:
        print(l.split(".")[-1], end=': ')
        print(l, end=', ')
        if i % 5 == 0:
            print("")

af: PAN-X.af, ar: PAN-X.ar, 
bg: PAN-X.bg, bn: PAN-X.bn, de: PAN-X.de, el: PAN-X.el, en: PAN-X.en, 
es: PAN-X.es, et: PAN-X.et, eu: PAN-X.eu, fa: PAN-X.fa, fi: PAN-X.fi, 
fr: PAN-X.fr, he: PAN-X.he, hi: PAN-X.hi, hu: PAN-X.hu, id: PAN-X.id, 
it: PAN-X.it, ja: PAN-X.ja, jv: PAN-X.jv, ka: PAN-X.ka, kk: PAN-X.kk, 
ko: PAN-X.ko, ml: PAN-X.ml, mr: PAN-X.mr, ms: PAN-X.ms, my: PAN-X.my, 
nl: PAN-X.nl, pt: PAN-X.pt, ru: PAN-X.ru, sw: PAN-X.sw, ta: PAN-X.ta, 
te: PAN-X.te, th: PAN-X.th, tl: PAN-X.tl, tr: PAN-X.tr, ur: PAN-X.ur, 
vi: PAN-X.vi, yo: PAN-X.yo, zh: PAN-X.zh, 

### 언어 선택하기

- 선택 언어
    - PAN-X.en : 영어
    - PAN-X.ko : 한국어
    - PAN-X.ja : 일본어
    - PAN-X.es : 스페인어

In [86]:
from datasets import load_dataset
from collections import defaultdict
from datasets import DatasetDict

#일반적인 불균형 상태 만들기
langs = ["en", "ko", "ja", "es"]
fracs = [0.12, 0.6, 0.8, 0.1]

panx_ch = defaultdict(DatasetDict)

In [90]:
for lang, frac in zip(langs, fracs):
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split]=(
            ds[split]
            .shuffle(seed=42)
            .select(range(int(frac*ds[split].num_rows))))


In [92]:
import pandas as pd

pd.DataFrame({lang : [panx_ch[lang]["train"].num_rows] for lang in langs}, index=["Samples"])

,en,ko,ja,es
Samples,2400,12000,16000,2000


In [93]:
element = panx_ch["ko"]["train"][0]

for k, v in element.items():
    print(k,v)

print("\nfeatures 중 ner-태그 확인")
ner_tags = panx_ch["ko"]["train"].features["ner_tags"].feature
ner_tags

tokens ['《트와일라잇》을', '같이', '찍은', '에디', '가테지', ',', '크리스틴', '스튜어트', ',', '로버트', '패틴슨과는', '매우', '친한사이라고', '한다', '.']
ner_tags [0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0]
langs ['ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko']

features 중 ner-태그 확인


ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [94]:
# NER-태그를 인덱스 에서 태그명 변경 및 추가

panx_ko = panx_ch["ko"].map(lambda x : {"ner_tag_names" :
                                        [ner_tags.int2str(idx)
                                        for idx in x["ner_tags"]]})
panx_ko

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 12000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 6000
    })
})

In [95]:
pd.DataFrame([panx_ko["train"][0]["tokens"],
              panx_ko["train"][0]["ner_tag_names"]],
              index=["tokens","ner_tag_name"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,《트와일라잇》을,같이,찍은,에디,가테지,",",크리스틴,스튜어트,",",로버트,패틴슨과는,매우,친한사이라고,한다,.
ner_tag_name,O,O,O,B-PER,I-PER,O,B-PER,I-PER,O,B-PER,I-PER,O,O,O,O


- 결과: 사람 이름에 태그가 붙었다.

### 태그 분포 확인하기

In [96]:
from collections import Counter

split2freqs = defaultdict(Counter)

for split , dataset in panx_ko.items():
    for row in dataset["ner_tag_names"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1

split2freqs

defaultdict(collections.Counter,
            {'train': Counter({'LOC': 7097, 'ORG': 5361, 'PER': 4870}),
             'validation': Counter({'LOC': 3579, 'ORG': 2755, 'PER': 2448}),
             'test': Counter({'LOC': 3503, 'ORG': 2584, 'PER': 2557})})

### 다중 언어 트랜스 포머

- 일반 적으로 데이터셋에 여러 언어가 있어도 일반화가 가능하다.
- 다중 언어 트랜스 포머 평가 방법
    1. **en (영어 훈련 후 평가)**: 영어 데이터로 훈련한 후 다른 언어에서 평가.
    2. **each (단일 훈련 및 단일 평가)**: 각 언어별로 별도로 훈련하고 평가.
    3. **all (모든 훈련셋에서 평가)**: 모든 언어 데이터를 사용해 훈련하고 각 언어에서 평가.

### XLM-R(Cross-lingual LM)
- RoBERTa 의 다중 언어 모델 버젼 : XLM-RoBERTa
- 100개 의 언어로 훈련

- **SentencePiece** 토크나이저
    - Unigram(부분 단어 분할 방식) 기반의 인코딩 방식을 이용
    - 특정 언어에 대한 지식없이 텍스트 처리 가능
    - 다국어 말뭉치에 유용하다
    - 다양한 언어 모델과 호환성이 좋다.

In [97]:
from transformers import AutoTokenizer

bertmodel = "bert-base-cased"
xlmrmodel = "xlm-roberta-base"

bert_tokenizer = AutoTokenizer.from_pretrained(bertmodel)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmrmodel) # Sentence Piece 토크나이저

In [98]:
text_test = "Im Your Father"

print(bert_tokenizer.convert_ids_to_tokens(bert_tokenizer(text_test).input_ids))
print(xlmr_tokenizer.convert_ids_to_tokens(xlmr_tokenizer(text_test).input_ids))

['[CLS]', 'I', '##m', 'Your', 'Father', '[SEP]']
['<s>', '▁Im', '▁Your', '▁Father', '</s>']


### 토큰화 파이프라인

1. **정규화 (Normalization)**
   - 텍스트를 일관된 형태로 변환.
   - 예: `'Im Your Father'` → `'im your father'`

2. **사전 토큰화 (Pre-tokenization)**
   - 공백과 구두점을 기준으로 단어로 나눈다..
   - 예: `'im your father'` → `['im', 'your', 'father']`

3. **토크나이저 모델 (Tokenizer Model)**
   - 단어를 고유한 숫자 ID로 바꿈.
   - 예: `['im', 'your', 'father']` → `[125, 52, 482]`

4. **사후처리 (Post-processing)**
   - 시작과 끝을 나타내는 특수 토큰을 추가.
   - 예: `[CLS] im your father [SEP]` → `[0, 125, 52, 482, 1]`

### 트랜스 포머 모델 클래스의 형식
![modelfortask](images/04_01.png)
- `<ModelName>For<Task>` 형식을 띔

In [99]:
import torch.nn as nn
from transformers import XLMRobertaXLConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel, RobertaPreTrainedModel

In [100]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaXLConfig

    def __init__(self, config):
        super().__init__(config) # 설정 초기화
        self.num_labels =  config.num_labels
        
        # 모델 바디 로드
        # add_pooling_layer=False : 모든 히든 스테이트 반환
        self.roberta = RobertaModel(config, add_pooling_layer=False)

        # 분류헤드
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # 가중치 로드 및 초기화
        self.init_weights()
    
    def forward(self, input_ids = None, attention_mask = None,
                token_type_ids = None, labels = None, **kwargs):
        # 모델 바디 순전파 결과
        outputs = self.roberta(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               **kwargs)
        
        # 분류 헤드 순전파
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)

        # 손실값 계산
        loss = None

        if labels is not None:
            loss_function = nn.CrossEntropyLoss()
            loss = loss_function(logits.view(-1, self.num_labels), labels.view(-1))

        # 객체 출력
        return TokenClassifierOutput(loss=loss, logits=logits,
                                     hidden_states=outputs.hidden_states,
                                     attentions=outputs.attentions)


In [101]:
# 태그 이름 인덱스 딕셔너리 생성
index2tag = {idx: tag for idx, tag in enumerate(ner_tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(ner_tags.names)}

In [102]:
from transformers import AutoConfig # 모델 구조의 설계도,

xlmr_config = AutoConfig.from_pretrained(xlmrmodel,
                                         num_labels = ner_tags.num_classes,
                                         id2label = index2tag,
                                         label2id = tag2index)

In [103]:
import torch

device = torch.device("cuda")
xlmr_model= (XLMRobertaForTokenClassification
               .from_pretrained(xlmrmodel, config=xlmr_config)
               .to(device))

In [104]:
input_ids = xlmr_tokenizer.encode(text_test, return_tensors="pt")
result_pd = pd.DataFrame([[int(i) for i in input_ids[0]],
              [xlmr_tokenizer.decode(i) for i in input_ids[0]]],
              index=["idx","token"])
result_pd

,0,1,2,3,4
idx,0,3370,14804,160960,2
token,<s>,Im,Your,Father,</s>


In [105]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs,dim=-1)

In [106]:
result_pd.loc["result"]=[index2tag[int(i)] for i in predictions[0]]

In [107]:
result_pd # 아직 훈련전 이기 때문에 

,0,1,2,3,4
idx,0,3370,14804,160960,2
token,<s>,Im,Your,Father,</s>
result,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC


In [108]:
### 이후 작업을 위하여 데이터 프레임 만들어주는 함수 생성

def tag_text(text, tags, model, tokenizer):
    # 텍스트 토크나이징
    tokens = tokenizer(text).tokens()
    # seq 인코딩
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    outputs = model(input_ids)[0]
    predictions = torch.argmax(outputs, dim=2)
    pred_str = [tags[p] for p in predictions[0]]
    return pd.DataFrame([tokens,
                         input_ids.cpu().numpy()[0],
                         pred_str], index=["Token","Ids","Tags"]
                        )


tag_text("transformer is fun!",
         ner_tags.names,
         xlmr_model,
         xlmr_tokenizer)

,0,1,2,3,4,5,6
Token,<s>,▁transform,er,▁is,▁fun,!,</s>
Ids,0,27198,56,83,7477,38,2
Tags,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC


In [109]:
words, tokens = panx_ko['train']['tokens'], panx_ko['train']['ner_tags']

In [110]:
tokenized_input = xlmr_tokenizer(words, is_split_into_words=True)

In [111]:
tokenized_input[0]

Encoding(num_tokens=44, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [112]:
i = 16
pd.DataFrame([tokenized_input[i].ids,
              tokenized_input[i].type_ids,
              tokenized_input[i].tokens,
              tokenized_input[i].offsets,
              tokenized_input[i].attention_mask,
              tokenized_input[i].special_tokens_mask,
              tokenized_input[i].overflowing],
              index=["input_ids", "type_ids", "tokens", "offsets", "attention_mask", "special_tokens_mask", "overflowing", ])

,0,1,2,3,4,5,6,7,8
input_ids,0,953,46349,19050,15,6,4,1388,2
type_ids,0,0,0,0,0,0,0,0,0
tokens,<s>,▁19,개의,▁지역,▁(,▁,",",▁),</s>
offsets,"(0, 0)","(0, 2)","(2, 4)","(0, 2)","(0, 1)","(0, 1)","(0, 1)","(0, 1)","(0, 0)"
attention_mask,1,1,1,1,1,1,1,1,1
special_tokens_mask,1,0,0,0,0,0,0,0,1
overflowing,None,None,None,None,None,None,None,None,None


### XLM-RoBERTa 파인 튜닝 하기

In [113]:
def tokens_to_input_ids(batch):
    return xlmr_tokenizer(batch['tokens'],
                          is_split_into_words=True, 
                          padding=True, 
                          truncation=True, 
                          return_tensors="pt")

# map 함수를 사용하여 데이터 변환
ko_train = panx_ko['train'].map(tokens_to_input_ids, batched=True, batch_size=12)
ko_valid = panx_ko['validation'].map(tokens_to_input_ids, batched=True, batch_size=12)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [114]:
from transformers import TrainingArguments

n_epoch = 3
b_size_ = 24
logging_steps = len(panx_ko['train']) // b_size_
model_name = f"{xlmrmodel}-finetuned-panx-kr"
train_arg = TrainingArguments(
    output_dir=model_name,
    log_level="error",
    num_train_epochs=n_epoch,
    per_device_eval_batch_size=b_size_,
    per_device_train_batch_size=b_size_,
    eval_strategy="epoch",
    save_steps=1e6,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=True,
    remove_unused_columns=False
)

In [115]:
from huggingface_hub import notebook_login

notebook_login()

In [116]:
from seqeval.metrics import f1_score
import numpy as np

# 예측 정리
def align_preds(preds, label_ids):
    preds = np.argmax(preds, axis=2)
    batch_size, seq_len = preds.shape
    labels_list , preds_list = [], []

    for batch_i in range(batch_size):
        example_labels, example_preds =[], []
        for seq_i in range(seq_len):
            if label_ids[batch_i,seq_i] != -100:
                example_labels.append(index2tag[label_ids[batch_i][seq_i]])
                example_preds.append(index2tag[preds[batch_i][seq_i]])
        labels_list.append(example_labels)
        preds_list.append(example_preds)

# 평가 결과
def result_metrics(eval_pred):
    y_pred, y_true = align_preds(eval_pred.predictions,
                                 eval_pred.label_ids)
    return {"f1" : f1_score(y_true, y_pred)}

- 데이터 콜레이터 : 토큰 분류 및 시퀀스 길이 패딩

In [117]:
from transformers import DataCollatorForTokenClassification

# xlmr 토크나이저를 이용하여 패딩 및 토크나이징
data_collator = DataCollatorForTokenClassification(xlmr_tokenizer, padding=True)

In [129]:
# 모델의 재사용을 위함 함수 정의
model = XLMRobertaForTokenClassification.from_pretrained(xlmrmodel, config=xlmr_config).to(device)